# Candlestick pattern modeling

### Import Library

In [15]:
import numpy as np
import pandas as pd
import seaborn as sns
import time
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
import requests

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120

import warnings
warnings.filterwarnings('ignore')

In [16]:
USER_AGENTS = [
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36"
]

HEADERS = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
VNDIRECT_DATA_HISTORY_URL = 'https://dchart-api.vndirect.com.vn/dchart/history'
URL = VNDIRECT_DATA_HISTORY_URL

In [17]:
def getStockHistoryData(ticker, timestamp_from=0, timestamp_to=0):
    if timestamp_from == 0:
        three_months = date.today() + relativedelta(months=-6)
        timestamp_from = datetime.strptime(three_months.strftime("%m/%d/%Y") + ', 00:00:0', "%m/%d/%Y, %H:%M:%S")\
            .timestamp()
    if timestamp_to == 0:
        timestamp_to = datetime.strptime(date.today().strftime("%m/%d/%Y") + ', 23:59:00', "%m/%d/%Y, %H:%M:%S")\
            .timestamp()

    params = {
        "resolution": "5",
        "symbol": str(ticker),
        "from": int(timestamp_from),
        "to": int(timestamp_to)
    }

    x = requests.get(URL, params=params, headers=HEADERS)
    response = x.json()

    import numpy as np
    import pandas as pd

    timestamp = np.array(response['t']).astype(int)
    close = np.array(response['c']).astype(float)
    open = np.array(response['o']).astype(float)
    high = np.array(response['h']).astype(float)
    low = np.array(response['l']).astype(float)
    volume = np.array(response['v']).astype(int)

    dataset = pd.DataFrame({'Time': timestamp, 'Open': list(open), 'High': list(high), 'Low': list(low),
                            'Close': list(close), 'Volume': list(volume)},
                           columns=['Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
    return dataset

In [18]:
def prepareData(htd):
    if 'Time' in htd.columns:
        from datetime import datetime

        htd['DateStr'] = htd.apply(
            lambda x: datetime.fromtimestamp(x['Time']).strftime("%Y-%m-%d %H:%M:%S"), axis=1)

    htd['Date'] = pd.to_datetime(htd['DateStr'])
    ticker_data = htd.set_index('Date')
    ticker_data.drop(columns=['Time', 'DateStr'], inplace=True)
    return ticker_data

### Load Price Data

In [19]:
ticker = "VN30F1M"
htd = getStockHistoryData(ticker, 0, 0)
ticker_data = prepareData(htd)
ticker_data = ticker_data.dropna()
len(ticker_data)

6220

In [20]:
import pspriceaction.price_action as pa

In [21]:
BACK_BARS=250
data = ticker_data.copy()
data = data.tail(BACK_BARS)

In [22]:
data["ma_line"] = data["Close"].rolling(20).mean()
data['above_ma'] = data.apply(lambda r: 1 if r['Close'] > r['ma_line'] else 0, axis=1)
data['below_ma'] = data.apply(lambda r: 1 if r['Close'] < r['ma_line'] else 0, axis=1)
data['total_above_ma_1w'] = data['above_ma'].rolling(250).sum()
data['total_below_ma_1w'] = data['below_ma'].rolling(250).sum()
data['trend'] = data.apply(lambda r: 'switch' if r['total_above_ma_1w'] == r['total_below_ma_1w'] else ('up' if r['total_above_ma_1w'] > r['total_below_ma_1w'] else 'down'), axis=1)

In [23]:
%%time
data = pa.pattern_modeling(data)

CPU times: user 329 ms, sys: 0 ns, total: 329 ms
Wall time: 328 ms


In [26]:
# data[data.model !=  '']

In [29]:
data[(data.index >  '2024-07-22 08:00:00') & (data.index <  '2024-07-22 15:00:00') & (data.model !=  '')]

,Open,High,Low,Close,Volume,ma_line,above_ma,below_ma,total_above_ma_1w,total_below_ma_1w,trend,color,min_OC,max_OC,upper_wick,tail,body,model
Date,,,,,,,,,,,,,,,,,,
2024-07-22 09:30:00,1296.0,1297.2,1295.8,1297.0,4888,1298.605,0,1,NaN,NaN,down,white,1296.0,1297.0,0.2,0.2,1.0,fair_value_rising_gap
2024-07-22 09:45:00,1297.7,1302.5,1297.7,1302.2,10399,1298.355,1,0,NaN,NaN,down,white,1297.7,1302.2,0.3,0.0,4.5,fair_value_rising_gap
2024-07-22 09:50:00,1302.3,1304.4,1301.7,1303.0,8873,1298.360,1,0,NaN,NaN,down,white,1302.3,1303.0,1.4,0.6,0.7,fair_value_rising_gap
2024-07-22 10:50:00,1296.8,1297.7,1295.8,1296.2,4881,1298.835,0,1,NaN,NaN,down,black,1296.2,1296.8,0.9,0.4,0.6,fair_value_falling_gap
2024-07-22 11:15:00,1293.6,1294.5,1292.5,1293.2,5334,1298.710,0,1,NaN,NaN,down,black,1293.2,1293.6,0.9,0.7,0.4,fair_value_falling_gap
2024-07-22 13:30:00,1295.7,1296.8,1295.0,1295.0,4748,1295.275,0,1,NaN,NaN,down,black,1295.0,1295.7,1.1,0.0,0.7,bullish_neck
2024-07-22 13:45:00,1293.4,1295.4,1293.4,1293.8,4289,1294.700,0,1,NaN,NaN,down,white,1293.4,1293.8,1.6,0.0,0.4,bearish_neck
